# Tutorial for the priority queueing network calculation (with priority discipline at nodes)
### Compare results with numerical calculations using decomposition method.

In [ ]:
# import libs
import numpy as np

from most_queue.general.tables import times_print_with_classes
from most_queue.rand_distribution import H2Distribution
from most_queue.sim.networks.priority_network import PriorityNetwork
from most_queue.theory.networks.open_network_prty import OpenNetworkCalcPriorities

### Set network parameters

In [2]:
k_num = 3  # Number of job classes
n_num = 5  # Number of network nodes

n = [3, 2, 3, 4, 3]  # Distribution of channels (servers) at each node
R = []  # List of transition probability matrices for each class
b = []  # List of service time moments for each class and node
for i in range(k_num):
    R.append(np.matrix([
        [1, 0, 0, 0, 0, 0],
        [0, 0.4, 0.6, 0, 0, 0],
        [0, 0, 0, 0.6, 0.4, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]
    ]))
L = [0.1, 0.3, 0.4]  # Arrival rates for each class
nodes_prty = []  # Priority distribution at each node: [node][class]

jobs_num = 200000  # Number of jobs simulated

utilization_for_node = 0.7  # Target utilization per node in the network
serve_cv = 1.2  # Coefficient of variation for service times (higher = more variability)

### Set servers parameters and nodes 
Use H<sub>2</sub> distribution for serving

In [3]:
serv_params = []

h2_params = []
for m in range(n_num):

    b1 = utilization_for_node * n[m] / sum(L)
    h2_params.append(H2Distribution.get_params_by_mean_and_coev(b1, serve_cv))

    serv_params.append([])
    for i in range(k_num):
        serv_params[m].append({'type': 'H', 'params': h2_params[m]})
        
for k in range(k_num):
    b.append([])
    for m in range(n_num):
        b[k].append(H2Distribution.calc_theory_moments(h2_params[m], 4))

### Set nodes priorities order nodes_prty
It looks like [m][x1, x2 .. x_k], where
- m - node number, 
- xi - priority for i-th class, 
- k - number of classes

### For example: 
- [0][0,1,2] - for the first node, a direct order of priorities is set,
- [2][0,2,1] - for the third node, such an order of priorities is set: for the first class - the oldest (0), for the second - the youngest (2), for the third - intermediate (1)



In [4]:
for m in range(n_num):
    nodes_prty.append([])
    for j in range(k_num):
        if m % 2 == 0:
            # Even-numbered nodes: priorities are assigned in ascending order (0, 1, 2, ...)
            nodes_prty[m].append(j)
        else:
            # Odd-numbered nodes: priorities are assigned in descending order (..., 2, 1, 0)
            nodes_prty[m].append(k_num - j - 1)

### Set priority discipline for each node
The service discipline in the node can be any from the list and is not necessarily the same for each node. 
In this case, we will set them to be the same and equal to NP.

Variants of priorities:
- No  - no priorities, FIFO
- PR  - preemptive resume, with resuming interrupted request
- RS  - preemptive repeat with resampling, re-sampling duration for new service
- RW  - preemptive repeat without resampling, repeating service with previous duration
- NP  - non preemptive, relative priority

In [5]:
prty = ['NP'] * n_num

### Run simulation

In [ ]:
# Create simulation of priority network:
qn = PriorityNetwork(k_num, L, R, n, prty, serv_params, nodes_prty)

#  Run simulation of priority network:
qn.run(jobs_num)

# Get initial moments of soujorney time from simulation:
v_sim    = qn.v_network

# Print sojourn initial moments for each class of priorities (3 moments for each class)
print(v_sim)

Job served:    | 0/100 [00:00<?, ?it/s]2000/200000:   1%|          | 1/100 [00:00<00:05, 17.36it/s]2000/200000:   2%|▏         | 2/100 [00:00<00:05, 16.69it/s]4000/200000:   2%|▏         | 2/100 [00:00<00:05, 16.69it/s]6000/200000:   3%|▎         | 3/100 [00:00<00:05, 16.69it/s]6000/200000:   4%|▍         | 4/100 [00:00<00:05, 16.99it/s]8000/200000:   4%|▍         | 4/100 [00:00<00:05, 16.99it/s]10000/200000:   5%|▌         | 5/100 [00:00<00:05, 16.99it/s]10000/200000:   6%|▌         | 6/100 [00:00<00:05, 17.01it/s]12000/200000:   6%|▌         | 6/100 [00:00<00:05, 17.01it/s]14000/200000:   7%|▋         | 7/100 [00:00<00:05, 17.01it/s]14000/200000:   8%|▊         | 8/100 [00:00<00:05, 17.08it/s]16000/200000:   8%|▊         | 8/100 [00:00<00:05, 17.08it/s]18000/200000:   9%|▉         | 9/100 [00:00<00:05, 17.08it/s]18000/200000:  10%|█         | 10/100 [00:00<00:05, 16.47it/s]20000/200000:  10%|█         | 10/100 [00:00<00:05, 16.47it/s]22000/200000:  11%|█         | 11/100 [00:00<00:05

[[9.595218209056053, 127.85105840368948, 2204.144958989117], [10.273703653474747, 144.4021869445579, 2577.5874583944287], [14.129456183837434, 288.31841750626967, 7685.83932712862]]


### Run calculation

In [ ]:
#  Get initial moments of soujorney time from calculation:
net_calc = OpenNetworkCalcPriorities(R, b, n, L, prty, nodes_prty)
net_calc = net_calc.run()
v_num = net_calc['v']
print(v_num)
# Get utilization factor of each node
loads = net_calc['loads']
print(
    f"Node utilization coefficients: {[float(round(load, 3)) for load in loads]}")

[[np.float64(9.654868738380173), np.float64(129.1076697107485), np.float64(2205.633897744974)], [np.float64(10.31289602913432), np.float64(145.7537506244222), np.float64(2612.8168517658423)], [np.float64(13.933273166903957), np.float64(279.9919985196908), np.float64(7344.270969156241)]]
Node utilization coefficients: [0.7, 0.28, 0.42, 0.168, 0.7]


### Print results for NP priority

In [ ]:
times_print_with_classes(v_sim, v_num, False)


       Initial moments of soujorn time in the system        
------------------------------------------------------------
           |               Number of moment                |
    Cls    | --------------------------------------------- |
           |       1       |       2       |       3       |
------------------------------------------------------------
     | Sim |      9.6      |      128      |    2.2e+03    |
  1  |------------------------------------------------------
     | Num |     9.65      |      129      |   2.21e+03    |
------------------------------------------------------------
     | Sim |     10.3      |      144      |   2.58e+03    |
  2  |------------------------------------------------------
     | Num |     10.3      |      146      |   2.61e+03    |
------------------------------------------------------------
     | Sim |     14.1      |      288      |   7.69e+03    |
  3  |------------------------------------------------------
     | Num |     13.9   

### Repeat all for PR priority

In [ ]:
prty = ['PR'] * n_num  # Absolute priority at each node
qn = PriorityNetwork(k_num, L, R, n, prty, serv_params, nodes_prty)
qn.run(jobs_num)
v_sim  = qn.v_network

net_calc = OpenNetworkCalcPriorities(R, b, n, L, prty, nodes_prty)
net_calc = net_calc.run()
v_num = net_calc['v']

times_print_with_classes(v_sim, v_num, False)


Job served:    | 0/100 [00:00<?, ?it/s]2000/200000:   1%|          | 1/100 [00:00<00:06, 14.28it/s]2000/200000:   2%|▏         | 2/100 [00:00<00:06, 15.19it/s]4000/200000:   2%|▏         | 2/100 [00:00<00:06, 15.19it/s]6000/200000:   3%|▎         | 3/100 [00:00<00:06, 15.19it/s]6000/200000:   4%|▍         | 4/100 [00:00<00:06, 15.59it/s]8000/200000:   4%|▍         | 4/100 [00:00<00:06, 15.59it/s]10000/200000:   5%|▌         | 5/100 [00:00<00:06, 15.59it/s]10000/200000:   6%|▌         | 6/100 [00:00<00:06, 15.57it/s]12000/200000:   6%|▌         | 6/100 [00:00<00:06, 15.57it/s]14000/200000:   7%|▋         | 7/100 [00:00<00:05, 15.57it/s]14000/200000:   8%|▊         | 8/100 [00:00<00:05, 15.47it/s]16000/200000:   8%|▊         | 8/100 [00:00<00:05, 15.47it/s]18000/200000:   9%|▉         | 9/100 [00:00<00:05, 15.47it/s]18000/200000:  10%|█         | 10/100 [00:00<00:05, 15.26it/s]20000/200000:  10%|█         | 10/100 [00:00<00:05, 15.26it/s]22000/200000:  11%|█         | 11/100 [00:00<00:05

       Initial moments of soujorn time in the system        
------------------------------------------------------------
           |               Number of moment                |
    Cls    | --------------------------------------------- |
           |       1       |       2       |       3       |
------------------------------------------------------------
     | Sim |     8.57      |      108      |   1.81e+03    |
  1  |------------------------------------------------------
     | Num |     8.54      |      107      |   1.75e+03    |
------------------------------------------------------------
     | Sim |     9.08      |      122      |   2.16e+03    |
  2  |------------------------------------------------------
     | Num |     10.1      |      140      |   2.51e+03    |
------------------------------------------------------------
     | Sim |     14.8      |      345      |   1.18e+04    |
  3  |------------------------------------------------------
     | Num |     15.9   